In [113]:
import torch
from torch import nn
import torch.nn.functional as F
import math
import numpy

In [114]:
def check(a,b):
    assert a.shape == b.shape, f'{a.shape} {b.shape}'
    assert torch.isclose(a,b.grad,atol=1e-6).all(), f'{a} {b.grad}'

In [161]:
B=20
C=4
H=50
CLASSES=3

y = torch.randint(0,2,size=(B,))
X = torch.randn((B,C))

w1 = torch.randn(C,H) / math.sqrt(C) / math.sqrt(2)
b1 = torch.zeros(H) 
w2 = torch.randn(H,H) / math.sqrt(H)  / math.sqrt(2) 
b2 = torch.zeros(H) 

proj_w = torch.randn(H,CLASSES)
proj_b = torch.zeros(CLASSES)

params = [w1,b1,w2,b2,proj_w,proj_b]

for p in params:
    p.requires_grad = True

In [179]:
dlogits*B

tensor([[-1.0708e-01,  9.3737e-02,  1.3343e-02],
        [-1.8898e-03,  1.6998e-03,  1.9001e-04],
        [ 7.1448e-06, -2.3603e-05,  1.6408e-05],
        [ 3.8724e-01, -4.1024e-01,  2.2998e-02],
        [ 9.0890e-02, -1.2328e-01,  3.2387e-02],
        [-1.8725e-01,  1.2580e-01,  6.1455e-02],
        [ 1.3942e-02, -1.8981e-02,  5.0392e-03],
        [-1.9257e-02,  1.2869e-02,  6.3874e-03],
        [ 3.3940e-02, -3.8430e-02,  4.4898e-03],
        [ 1.1845e-01, -1.3141e-01,  1.2960e-02],
        [-3.5583e-01,  3.0753e-01,  4.8291e-02],
        [-5.6557e-02,  4.9081e-02,  7.4759e-03],
        [ 1.9431e-01, -2.0716e-01,  1.2853e-02],
        [ 2.6587e-02, -3.6173e-02,  9.5860e-03],
        [-2.2896e-01,  1.9981e-01,  2.9153e-02],
        [-1.4738e-01,  1.3204e-01,  1.5335e-02],
        [-1.0942e-01,  1.0328e-01,  6.1402e-03],
        [ 3.8698e-03, -7.6649e-03,  3.7952e-03],
        [-4.1177e-02,  3.9461e-02,  1.7153e-03],
        [-1.0427e-02,  8.9497e-03,  1.4770e-03]])

In [182]:
y

tensor([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0])

In [180]:
1-probas

tensor([[1.0708e-01, 9.0626e-01, 9.8666e-01],
        [1.8898e-03, 9.9830e-01, 9.9981e-01],
        [9.9999e-01, 2.3603e-05, 9.9998e-01],
        [6.1276e-01, 4.1024e-01, 9.7700e-01],
        [9.0911e-01, 1.2328e-01, 9.6761e-01],
        [1.8725e-01, 8.7420e-01, 9.3855e-01],
        [9.8606e-01, 1.8981e-02, 9.9496e-01],
        [1.9257e-02, 9.8713e-01, 9.9361e-01],
        [9.6606e-01, 3.8430e-02, 9.9551e-01],
        [8.8155e-01, 1.3141e-01, 9.8704e-01],
        [3.5583e-01, 6.9247e-01, 9.5171e-01],
        [5.6557e-02, 9.5092e-01, 9.9252e-01],
        [8.0569e-01, 2.0716e-01, 9.8715e-01],
        [9.7341e-01, 3.6173e-02, 9.9041e-01],
        [2.2896e-01, 8.0019e-01, 9.7085e-01],
        [1.4738e-01, 8.6796e-01, 9.8467e-01],
        [1.0942e-01, 8.9672e-01, 9.9386e-01],
        [9.9613e-01, 7.6649e-03, 9.9620e-01],
        [4.1177e-02, 9.6054e-01, 9.9828e-01],
        [1.0427e-02, 9.9105e-01, 9.9852e-01]], grad_fn=<RsubBackward1>)

In [176]:
for i in range(10):
    # forward
    pre1 = X@w1+b1
    act1 = torch.relu(pre1)
    pre2 = act1@w2+b2
    act2 = torch.relu(pre2)
    logits = act2@proj_w+proj_b
    
    # probas = torch.softmax(logits, 1)
    logits_minus_max = logits-logits.max()
    logits_exp = torch.exp(logits_minus_max)
    denom = logits_exp.sum(-1,keepdims=True)
    probas = logits_exp / denom
    
    sel_probas = probas[range(len(y)), y]
    log_probs = torch.log(sel_probas)
    loss = -log_probs.mean()
    print(f'{loss.item()}')
    # log_probs.requires_grad=True
    
    vars = [pre1,act1,pre2,act2,logits,probas,sel_probas,log_probs,logits_exp,denom,logits_minus_max]
    for v in vars:
        v.retain_grad()
    
    for p in vars + params:
        p.grad = None
    
    loss.backward()
    
    with torch.no_grad():
        # dlog_probs = torch.ones_like(log_probs)*-1/B
        # dsel_probas = 1/sel_probas * dlog_probs
        # dprobas = torch.zeros_like(logits)
        # dprobas[range(len(y)), y] = dsel_probas
        # dlogits_exp = 1/denom * dprobas
        # ddenom = (-denom**-2 * logits_exp * dprobas).sum(1,keepdims=True)
        # dlogits_exp = 1/denom * dprobas
        # dlogits_exp += ddenom    
        # dlogits_minus_max = dlogits_exp * logits_exp
        # dlogits = dlogits_minus_max

        dlogits = probas.clone()
        dlogits[range(B), y] -= 1
        dlogits /= B
        
        dact2 = dlogits@proj_w.T
        dproj_w = act2.T@dlogits    
        dproj_b = dlogits.sum(0)
        dpre2 = torch.where(pre2>0, dact2, 0)
        check(dpre2, pre2)
        dact1 = dpre2@w2.T
        dw2 = act1.T@dpre2
        db2 = dpre2.sum(0)
        dpre1 = torch.where(pre1>0,dact1,0)
        dw1 = X.T@dpre1
        db1 = dpre1.sum(0)
    
    dparams = [dw1,db1,dw2,db2,dproj_w,dproj_b]
    
    lr=1e-2
    for p,dp in zip(params,dparams):
        check(dp,p)
        assert dp.shape == p.shape
        # ass
        with torch.no_grad():
            p.data -= lr*dp



0.13677462935447693
0.13573968410491943
0.13477015495300293
0.13373026251792908
0.13279816508293152
0.13181805610656738
0.1308976113796234
0.12997914850711823
0.12904365360736847
0.12816651165485382


tensor([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0])